In [23]:
import os 

import requests
import shutil
import json

import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
base_url = 'https://pixabay.com/api/?key='
TOKEN = '24807314-8517bb66442790f99df3f9cec'
q = 'frog'
per_page = 200 
max_page = 5

In [17]:
# Accepts request parameters as input and constructs a request to the pixabay service
def make_query_url(base_url=base_url,
                   q=q,
                   TOKEN=TOKEN,
                   per_page=per_page, 
                   page=1): 
    query = f'{base_url + TOKEN + "&q="}'

    return query

In [9]:
path = './images'

In [10]:
# Accepts an image url as input and stores it in the path
def get_image(url, path=path):
  broken_images=[]
  for img in url:
  # We can split the file based upon / and extract the last split within the python list below:
      save_path = './images/animal'
      file_name = img.split('/')[-1]
      completeName = os.path.join(save_path, file_name)
      print(f"This is the file name: {file_name}")
  # Now let's send a request to the image URL:
      r = requests.get(img, stream=True)
      # We can check that the status code is 200 before doing anything else:
      if r.status_code == 200:
      # This command below will allow us to write the data to a file as binary:
          with open(completeName, 'wb') as f:
              for chunk in r:
                  f.write(chunk)
      else:
      # We will write all of the images back to the broken_images list:
          broken_images.append(img)

In [18]:
# Accepts (request, search results with pictures will be generated by presence), per_page (number of images in one output), max_page (how many pages to sort through)
url_list = []
def parse(q, per_page, max_page):
    
    query = make_query_url()+ q + "&per_page=" + str(per_page) + "&page=" + str(max_page)
    response = requests.get(make_query_url(), stream=True)
    r = json.loads(requests.get(query).content)
    r_=r['hits']
    for i in r_:
        url_list.append(i['previewURL'])
    return url_list

In [19]:
get_image(parse(q='frog', per_page=200, max_page=2))

This is the file name: kawaii-frogs-5102688_150.png
This is the file name: poison-dart-frog-5487620_150.png
This is the file name: frog-1672887_150.jpg
This is the file name: frog-3869456_150.jpg
This is the file name: tree-frog-324553_150.jpg
This is the file name: frog-5709420_150.png
This is the file name: hang-out-1521445_150.jpg
This is the file name: kawaii-frogs-5102686_150.png
This is the file name: frog-193944_150.jpg
This is the file name: frog-4454401_150.png
This is the file name: frog-46396_150.png
This is the file name: amphibian-1297311_150.png
This is the file name: frog-310436_150.png
This is the file name: nature-2727199_150.jpg
This is the file name: frog-king-311629_150.png
This is the file name: turtle-4707975_150.png
This is the file name: animal-793003_150.png
This is the file name: frog-1519004_150.jpg
This is the file name: red-eye-frog-2815682_150.png
This is the file name: fantasy-6611542_150.jpg
This is the file name: relaxation-1715385_150.jpg
This is the f

In [20]:
print(parse(q='pig', per_page=200, max_page=2))

['https://cdn.pixabay.com/photo/2020/04/28/04/29/kawaii-frogs-5102688_150.png', 'https://cdn.pixabay.com/photo/2020/08/14/11/08/poison-dart-frog-5487620_150.png', 'https://cdn.pixabay.com/photo/2016/09/15/22/53/frog-1672887_150.jpg', 'https://cdn.pixabay.com/photo/2018/12/11/18/41/frog-3869456_150.jpg', 'https://cdn.pixabay.com/photo/2014/04/15/02/18/tree-frog-324553_150.jpg', 'https://cdn.pixabay.com/photo/2020/11/03/10/58/frog-5709420_150.png', 'https://cdn.pixabay.com/photo/2016/07/16/12/34/hang-out-1521445_150.jpg', 'https://cdn.pixabay.com/photo/2020/04/28/04/28/kawaii-frogs-5102686_150.png', 'https://cdn.pixabay.com/photo/2013/10/11/06/39/frog-193944_150.jpg', 'https://cdn.pixabay.com/photo/2019/09/05/15/42/frog-4454401_150.png', 'https://cdn.pixabay.com/photo/2012/05/02/21/29/frog-46396_150.png', 'https://cdn.pixabay.com/photo/2016/03/31/22/58/amphibian-1297311_150.png', 'https://cdn.pixabay.com/photo/2014/04/03/10/26/frog-310436_150.png', 'https://cdn.pixabay.com/photo/2017/09/

In [21]:
# Image folder cleanup function
def clear_dir(path):
    import os
    files = os.listdir(path)
    print(files)
    [os.remove(f'{path+f}') for f in files if 'png' in f]

## Image classification with pre-trained models

In [24]:
model = InceptionResNetV2(weights='imagenet')

225222656/225209952 [==============================] - 1s 0us/step


In [25]:
data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [27]:
data = data_gen.flow_from_directory('/content/drive/MyDrive/Colab_Notebooks/pixabay/images')

Found 200 images belonging to 2 classes.


In [28]:
predictions = model.predict(data)

In [29]:
count = 0
for i in np.arange(len(decode_predictions(predictions, top=1))):
    possible_labels = ['toad', 'frog'] 
    decoded = decode_predictions(predictions, top=1)[i][0][1]
    if any(label in decoded for label in possible_labels):
        count += 1


49152/35363 [=========================================] - 0s 0us/step


In [31]:
count / 200

0.65